# Meridional Overturning Circulation Notebook (rho-space)

This notebook plots the global MOC in isopycnal (rho2) space.

Authors: Jan-Erik Tesdal and John Krasting

## Framework Code and Diagnostic Setup

In [ ]:
import esnb
from esnb import CaseGroup2, NotebookDiagnostic, RequestedVariable, nbtools
from esnb.sites.gfdl import call_dmget

In [ ]:
# Define a mode (leave "prod" for now)
mode = "prod"

# Verbosity
verbose = True

# Give your diagnostic a name and a short description
diag_name = "MOC rho-space"
diag_desc = "Meridional Overturning Circulation in rho-space"

# Define what variables you would like to analyze. The first entry is the
# variable name and the second entry is the realm (post-processing dir).
#   (By default, monthly timeseries data will be loaded. TODO: add documentation
#    on how to select different frequencies, multiple realms to search, etc.)
variables = [
    RequestedVariable("vmo", "ocean_annual_rho2", frequency="yearly"),
]

# Optional: define runtime settings or options for your diagnostic
user_options = {"regions": ["global"]}

# Initialize the diagnostic with its name, description, vars, and options
diag = NotebookDiagnostic(diag_name, diag_desc, variables=variables, **user_options)

In [ ]:
# Define the groups of experiments to analyze. Provide a single dora id for one experiment
# or a list of IDs to aggregate multiple experiments into one; e.g. historical+future runs
groups = [
    CaseGroup2("odiv-516", date_range=("1993-01-01", "2017-12-31"), name="OM5 B11 NB"),
    CaseGroup2(
        "odiv-290", date_range=("1993-01-01", "2017-12-31"), name="OM5 B01 (OM4-like)"
    ),
    CaseGroup2(
        "odiv-554", date_range=("0041-01-01", "0060-12-31"), name="CM4.0 + OM5 B11"
    ),
    CaseGroup2("odiv-558", date_range=("0041-01-01", "0060-12-31"), name="CM4.0"),
]

In [ ]:
# Combine the experiments with the diag request and determine what files need to be loaded:
diag.resolve(groups)

In [ ]:
# Print a list of file paths
# This cell and the markdown cell that follows are necessary to run this notebook
# Interactively on Dora
_ = [print(x) for x in diag.files]

<i>(The files above are necessary to run the diagnostic.)</i>

In [ ]:
# Check to see the dmget status before calling "open"
call_dmget(diag.files,status=True)
call_dmget(diag.files)

In [ ]:
# Load the data as xarray datasets
diag.open()

## Main Diagnostic

In [ ]:
import cmip_basins
import matplotlib.pyplot as plt
import momgrid as mg
import numpy as np
import xarray as xr

In [ ]:
# Capture interfaces to use later
interfaces = [xr.DataArray(x["rho2_i"]) for x in diag.datasets]

In [ ]:
esnb.sites.gfdl.convert_to_momgrid(diag)

In [ ]:
def calc_moc_rho(
    ds,
    varname,
    interfaces,
    mask=1.0,
    xdim="xh",
    tdim="time",
    ydim="yq",
    zdim="rho2_l",
    rho0=1035.0,
):
    arr = ds[varname]
    arr = arr.where(arr < 1.0e14)
    arr = arr.mean(tdim) * mask
    integ_layers = (arr.sum(xdim).cumsum(zdim) - arr.sum(xdim).sum(zdim)) / rho0 / 1.0e6
    bottom_condition = xr.zeros_like(integ_layers.isel({zdim: 0}))
    psi_raw = xr.concat([bottom_condition, integ_layers], dim=zdim)
    psi = psi_raw.rename({zdim: interfaces.name}).transpose(interfaces.name, ydim)
    psi[interfaces.name] = interfaces  # psi.name = "psi"

    y = (ds.geolat_v * mask).mean(xdim)
    y = xr.DataArray(y.values, dims=("y"), coords={"y": y.values})

    return (psi, y)

In [ ]:
moc_rho_data = {}
varname = "vmo"
varobj = diag.varmap[varname]
xdim = "xh"
ydim = "yq"
zdim = "rho2_l"
tdim = "time"

moc_rho_data = {}

for region in diag.settings["diag_vars"]["regions"]:
    moc_rho_data[region] = {}
    for n, group in enumerate(diag.groups):
        print(f"Calculating {region} moc rho for {group.name}")
        ds = group.datasets[varobj]
        mask = 1.0
        psi, y = calc_moc_rho(ds, varname, interfaces[n], mask=mask)
        psi = psi.load()
        moc_rho_data[region][group] = (psi, y)

In [ ]:
all_figs = []

esnb.nbtools.setup_plots()

for region in diag.settings["diag_vars"]["regions"]:
    nexps = len(diag.groups)
    figsize, subplot = esnb.nbtools.get_figsize_subplots(nexps)
    fig = plt.figure(figsize=figsize, dpi=200)

    axes = []

    for n, group in enumerate(diag.groups):
        ax = plt.subplot(*subplot, n + 1)
        stats = {}

        psi, y = moc_rho_data[region][group]
        levels = np.arange(-40, 42, 2)

        cb = ax.contourf(y, psi.rho2_i, psi, levels=levels, cmap="RdBu_r")
        cs = ax.contour(y, psi.rho2_i, psi, levels=levels, colors="k", linewidths=0.3)

        r = psi.rho2_i
        Y, R = np.meshgrid(y, r)
        mask = (Y >= -75) & (Y <= -50) & (R >= 1036.8) & (R <= 1037.4)
        _psi = psi.values
        min_psi = np.min(_psi[mask])
        min_index = np.unravel_index(np.argmin(_psi[mask]), _psi[mask].shape)
        min_y = Y[mask][min_index]
        min_r = R[mask][min_index]

        stats = {
            "min_psi": round(float(min_psi), 2),
            "min_lat": round(float(min_y), 2),
            "min_rho": round(float(min_r), 2),
        }

        square_size = 10
        ax.plot(
            min_y,
            min_r,
            marker="s",
            color="magenta",
            markersize=square_size + 5,
            markeredgewidth=2,
            markeredgecolor="magenta",
            markerfacecolor="none",
        )
        ax.annotate(
            f"{min_psi:.1f} Sv",
            xy=(min_y, min_r),
            xytext=(10, 10),
            textcoords="offset points",
            arrowprops=dict(arrowstyle="->", color="black"),
            bbox=dict(
                facecolor="white",
                edgecolor="black",
                boxstyle="round,pad=0.2",
                linewidth=0.5,
                alpha=0.7,
            ),
        )

        ax.set_yscale("splitscale", zval=[1037.25, 1036.5, 1028.8])
        ax.set_title(group.name)
        axes.append(ax)

        if len(stats) > 0:
            for k, v in stats.items():
                group.add_metric(region, (k, float(v)))

        if region == "global":
            for ax in axes:
                ax.set_xlim(-78, None)

    plt.subplots_adjust(wspace=0.3)

    cbar = esnb.nbtools.bottom_colorbar(
        fig, cb, orientation="horizontal", extend="both"
    )
    cbar.set_label(f"{str(region).capitalize()} Meridional Streamfunction [Sv]")

    # add letter labels for each panel
    esnb.nbtools.panel_letters(axes, -0.12, 1.05)

In [ ]:
all_figs = []

esnb.nbtools.setup_plots()

for region in diag.settings["diag_vars"]["regions"]:
    nexps = len(diag.groups)
    figsize, subplot = esnb.nbtools.get_figsize_subplots(nexps)
    figsize = (5.41 * 1.25, 3.35 * 0.75 * 2)
    fig = plt.figure(figsize=figsize, dpi=200)

    axes = []
    for n, group in enumerate(diag.groups):
        ax = plt.subplot(*subplot, n + 1)
        stats, cb = plot_panel(ax, *moc_z_data[region][group])
        ax.set_title(group.name)
        axes.append(ax)

        if len(stats) > 0:
            for k, v in stats.items():
                group.add_metric(region, (k, float(v)))

    if region == "global":
        for ax in axes:
            ax.set_xlim(-78, None)

    plt.subplots_adjust(wspace=0.3)

    cbar = esnb.nbtools.bottom_colorbar(
        fig, cb, orientation="horizontal", extend="both"
    )
    cbar.set_label(f"{str(region).capitalize()} Meridional Streamfunction [Sv]")

    # add letter labels for each panel
    esnb.nbtools.panel_letters(axes, -0.12, 1.05)

In [ ]:
diag.write_metrics("MOC_rho_metrics.json")